In [18]:
from easyAI import TwoPlayerGame, Human_Player, AI_Player, Negamax
import pygame, sys
pygame.init()

# global variables
WIDTH, HEIGHT = 512, 512
MESSAGE_MARGIN = 30
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
COLORS = [
    (60, 32, 16), # brown
    (16, 144, 85), # green
    (199, 69, 50), # red
    (229, 201, 33), # yellow
    (217, 121, 170), # pink
    (103, 55, 134), # purple
    (5, 107, 173), # blue
    (214, 116, 33) # orange
]
BOARD = [
    [ 7, 6, 5, 4, 3, 2, 1, 0 ],
    [ 2, 7, 4, 1, 6, 3, 0, 5 ],
    [ 1, 4, 7, 2, 5, 0, 3, 6 ],
    [ 4, 5, 6, 7, 0, 1, 2, 3 ],
    [ 3, 2, 1, 0, 7, 6, 5, 4 ],
    [ 6, 3, 0, 5, 2, 7, 4, 1 ],
    [ 5, 0, 3, 6, 1, 4, 7, 2 ],
    [ 0, 1, 2, 3, 4, 5, 6, 7 ]
]
PLAYERCOLOR = [ BLACK, WHITE ]
BLOCK_DELAY = 5

# set pygame
font = pygame.font.SysFont("comicsans",20)
screen = pygame.display.set_mode((WIDTH, HEIGHT + MESSAGE_MARGIN))
pygame.display.set_caption("KAMISADO")

class Kamisado(TwoPlayerGame):
    # First of all, the definition of what a game needs.
    def __init__(self, players):
        self.players = players
        self.current_player = 1

        self.first_move = True
        # original
        self.tower = [
            [ (0, 7), (0, 6), (0, 5), (0, 4), (0, 3), (0, 2), (0, 1), (0, 0) ],
            [ (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7) ]
        ]
        # # block
        # self.tower = [
        #     [ (5, 7), (6, 6), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0) ],
        #     [ (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7) ]
        # ]
        # # dead lock
        # self.tower = [
        #     [ (2, 5), (3, 6), (5, 7), (2, 4), (3, 3), (2, 0), (2, 1), (2, 2) ],
        #     [ (3, 1), (3, 0), (3, 2), (4, 3), (3, 4), (3, 5), (5, 4), (7, 7) ]
        # ]
        self.turn = 1
        self.tower_to_move = -1
        self.tower_to_move_coord = None

        self.last_player_before_blocked = None
        self.blocked_coord_list = None

        self.is_win = False

    # Defines all possible movements on the board.
    def possible_moves(self):
        print("언제실행되 파시블무브>??")
        result = []
        forward = 1 if self.turn == 0 else -1
        print(self.turn)
        print(self.tower_to_move)
        row, col = self.tower[self.turn][self.tower_to_move]
        cur_tower_list = self.tower[0] + self.tower[1]
        print(cur_tower_list)
        
        # strait moves
        for i in range(1, 8):
            if forward == 1 and row + i * forward > 7: break
            elif forward == -1 and row + i * forward < 0: break
            if (row + i * forward, col) in cur_tower_list: break
            else: result.append((row + i * forward, col))

        # left diagonal moves
        for i in range(1, 8):
            if forward == 1 and (row + i * forward > 7 or col + i * forward > 7): break
            elif forward == -1 and (row + i * forward < 0 or col + i * forward < 0): break
            if (row + i * forward, col + i * forward) in cur_tower_list: break
            else: result.append((row + i * forward, col + i * forward))

        # right diagonal moves
        for i in range(1, 8):
            if forward == 1 and (row + i * forward > 7 or col - i * forward < 0): break
            elif forward == -1 and (row + i * forward < 0 or col - i * forward > 7): break
            if (row + i * forward, col - i * forward) in cur_tower_list: break
            else: result.append((row + i * forward, col - i * forward))
        
        # when tower is blocked
        if not result: result.append((row, col))
        
        print(result,"result")
        return result
    
    # Define how to update the board after the player moves.
    def make_move(self, move):
        print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
        print(move) 

        self.tower[self.turn][self.tower_to_move] = move
        print("tower update")
        # check if any player win
        if not self.is_over():
            self.set_next_turn(move)
        
        # check if next player is blocked
        if self.is_blocked():
            # check for deadlock
            if self.is_deadlock(self.turn, self.tower_to_move, self.tower_to_move_coord):
                self.is_win = True
    
    def is_blocked(self, coord=None, turn=None):
        coord = coord if coord is not None else self.tower_to_move_coord
        turn = turn if turn is not None else self.turn
        forward = 1 if turn == 0 else -1
        towers = self.tower[0] + self.tower[1]

        c1 = (coord[0] + forward, coord[1]) in towers
        c2 = (coord[0] + forward, 0 if coord[1] - 1 < 0 else coord[1] - 1) in towers
        c3 = (coord[0] + forward, 7 if coord[1] + 1 > 7 else coord[1] + 1) in towers

        if c1 and c2 and c3: return True
        else: return False

    def is_deadlock(self, turn, tower_to_move, tower_to_move_coord):
        turn = 1 if turn == 0 else 0
        tower_to_move = BOARD[tower_to_move_coord[0]][tower_to_move_coord[1]]
        tower_to_move_coord = self.tower[turn][tower_to_move]
        if self.is_blocked(tower_to_move_coord, turn):
            if tower_to_move_coord not in self.blocked_coord_list:
                self.blocked_coord_list.append(tower_to_move_coord)
                self.is_deadlock(turn, tower_to_move, tower_to_move_coord)
            # deadlock occurs!
            else: return True
        else: return False
    
    def set_next_turn(self, coord):
        self.blocked_coord_list = [ coord ]
        self.last_player_before_blocked = self.turn

        # switch turn
        self.turn = 1 if self.turn == 0 else 0

        # move tower
        self.tower_to_move = BOARD[coord[0]][coord[1]]
        self.tower_to_move_coord = self.tower[self.turn][self.tower_to_move]
        self.first_move = False

    # check if the game is over
    def is_over(self):        
        return (
            self.is_win
            or self.win(self.current_player - 1)
            or self.win(self.opponent_index - 1)
        )

    def scoring(self):
        opp_won = self.win(self.opponent_index - 1)
        i_won = self.win(self.current_player - 1)
        if opp_won and not i_won:
            return 100
        elif i_won and not opp_won:
            return -100
        else:
            return 0

    def win(self, who):
        end_line_index = 7 if who == 0 else 0
        for i in self.tower[who]:
            if i[0] == end_line_index:
                return True
        return False

    def select_tower_to_move_first(self, coord):
        for i, t in enumerate(self.tower[self.turn]):
            if coord == t:
                self.tower_to_move = i
                self.tower_to_move_coord = coord

    def reset_tower_to_move_first(self):
        self.tower_to_move = -1
            

def draw_grid():
    for i in range(8): # index of row
        for j in range(8): # index of col
            # locate a rect at i-th row and j-th col
            rect = pygame.Rect(
                (j * WIDTH/8, i * HEIGHT/8), (WIDTH/8, HEIGHT/8))
            pygame.draw.rect(screen, COLORS[BOARD[i][j]], rect, width=0)

def draw_tower(game):
    for p in range(2): # player (0 or 1)
        for i in range(8): # each tower
            center = (game.tower[p][i][1] * WIDTH/8 + WIDTH/16,
                        game.tower[p][i][0] * HEIGHT/8 + HEIGHT/16)
            pygame.draw.circle(screen, COLORS[i], center, WIDTH/16, width=0)
            pygame.draw.circle(screen, PLAYERCOLOR[p], center, WIDTH/16, width=2)

def _message_margin(txt, backgr, textcol):
    rect = pygame.Rect( (0, HEIGHT), (WIDTH, MESSAGE_MARGIN) )
    pygame.draw.rect(screen, backgr, rect)
    pos_text = font.render(txt, True, textcol)
    pos_rect = pos_text.get_rect()
    pos_rect.center = ( WIDTH/2, HEIGHT + MESSAGE_MARGIN/2 )
    screen.blit(pos_text, pos_rect)

def msg_turn(game):
    title = "Win" if game.is_over() else "Turn"

    if game.turn == 0: # turn of blacks
        txt = "{}: {}".format(title, 'black')
        _message_margin(txt, BLACK, WHITE)
    else:
        txt = "{}: {}".format(title, 'white')
        _message_margin(txt, WHITE, BLACK)

def msg_blocked(game):
    if game.turn == 0: # turn of blacks
        txt = "Blocked: next turn -> {} in {} secs".format('white', BLOCK_DELAY)
        _message_margin(txt, BLACK, WHITE)
    else:
        txt = "Blocked: next turn -> {} in {} secs".format('black', BLOCK_DELAY)
        _message_margin(txt, WHITE, BLACK)

def highlight_cell(game):
    if game.tower_to_move >= 0:
        rect = pygame.Rect(
                    (game.tower_to_move_coord[1] * WIDTH/8, 
                    game.tower_to_move_coord[0] * HEIGHT/8),
                    (WIDTH/8, HEIGHT/8))
        pygame.draw.rect(screen, PLAYERCOLOR[game.turn], rect, width=2)

def calc_coord(pos):
    
    return (int(pos[1] / (HEIGHT/8)), int(pos[0] / (WIDTH/8)))

def draw_board(game):
    draw_grid()
    draw_tower(game)
    msg_turn(game)
    highlight_cell(game)
    pygame.display.update()

if __name__ == '__main__':
    kamisado = Kamisado([Human_Player(), AI_Player(Negamax(1))])
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                print("이벤트")
                coord = calc_coord(pygame.mouse.get_pos())

                if kamisado.tower_to_move < 0:
                    kamisado.select_tower_to_move_first(coord)
                    continue

                if kamisado.first_move and kamisado.tower_to_move_coord == coord:
                    kamisado.reset_tower_to_move_first()
                
                # MY TURN
                elif coord in kamisado.possible_moves():
                    print("aaaaaaaaaaaaaaaaaaaaaaa")
                    #print("coord가 result에 있을까?",coord)
                    kamisado.play_move(coord)
                    
                    print("이동함")
                
                    if kamisado.is_over():
                        print(kamisado.tower_to_move_coord)
                        kamisado.tower_to_move_coord = coord
                        print("끝났네")
                        print(coord)

                    # AI TURN
                    else:
                        print("dsf")
                        draw_board(kamisado)
                        print("sss")
                        pygame.time.delay(500)
                        ai_move = kamisado.get_move()
                    
                        kamisado.play_move(ai_move)
                        print("ai도 make move하나")

                        if kamisado.is_over():
                            kamisado.tower_to_move_coord = ai_move
                        
        draw_board(kamisado)
 

이벤트
이벤트
언제실행되 파시블무브>??
1
4
[(0, 7), (0, 6), (0, 5), (0, 4), (0, 3), (0, 2), (0, 1), (0, 0), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)]
[(6, 4), (5, 4), (4, 4), (3, 4), (2, 4), (1, 4), (6, 3), (5, 2), (4, 1), (3, 0), (6, 5), (5, 6), (4, 7)] result
aaaaaaaaaaaaaaaaaaaaaaa
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
(3, 4)
tower update
이동함
dsf
sss
언제실행되 파시블무브>??
0
0
[(0, 7), (0, 6), (0, 5), (0, 4), (0, 3), (0, 2), (0, 1), (0, 0), (7, 0), (7, 1), (7, 2), (7, 3), (3, 4), (7, 5), (7, 6), (7, 7)]
[(1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (1, 6), (2, 5)] result
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
(1, 7)
tower update
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
(2, 7)
tower update
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
(3, 7)
tower update
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
(4, 7)
tower update
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
(5, 7)
tower update
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
(6, 7)
tower update
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
(1, 6)
tower update
$$$$$$$$$$$$$$$$$$$$$$$$$$$$

SystemExit: 

In [27]:
rr=[(1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (1, 6), (2, 7), (1, 4)]
if not rr:
    print("D")

In [16]:
from easyAI import TwoPlayerGame, Human_Player, AI_Player, Negamax
import pygame, sys
pygame.init()

# global variables
WIDTH, HEIGHT = 512, 512
MESSAGE_MARGIN = 30
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
COLORS = [
    (60, 32, 16), # brown
    (16, 144, 85), # green
    (199, 69, 50), # red
    (229, 201, 33), # yellow
    (217, 121, 170), # pink
    (103, 55, 134), # purple
    (5, 107, 173), # blue
    (214, 116, 33) # orange
]
BOARD = [
    [ 7, 6, 5, 4, 3, 2, 1, 0 ],
    [ 2, 7, 4, 1, 6, 3, 0, 5 ],
    [ 1, 4, 7, 2, 5, 0, 3, 6 ],
    [ 4, 5, 6, 7, 0, 1, 2, 3 ],
    [ 3, 2, 1, 0, 7, 6, 5, 4 ],
    [ 6, 3, 0, 5, 2, 7, 4, 1 ],
    [ 5, 0, 3, 6, 1, 4, 7, 2 ],
    [ 0, 1, 2, 3, 4, 5, 6, 7 ]
]
PLAYERCOLOR = [ BLACK, WHITE ]
BLOCK_DELAY = 5

# set pygame
font = pygame.font.SysFont("comicsans",20)
screen = pygame.display.set_mode((WIDTH, HEIGHT + MESSAGE_MARGIN))
pygame.display.set_caption("KAMISADO")

class Kamisado(TwoPlayerGame):
    # First of all, the definition of what a game needs.
    def __init__(self, players):
        self.players = players
        self.current_player = 1

        self.first_move = True
        # original
        self.tower = [
            [ (0, 7), (0, 6), (0, 5), (0, 4), (0, 3), (0, 2), (0, 1), (0, 0) ],
            [ (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7) ]
        ]
        # # block
        # self.tower = [
        #     [ (5, 7), (6, 6), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0) ],
        #     [ (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7) ]
        # ]
        # # dead lock
        # self.tower = [
        #     [ (2, 5), (3, 6), (5, 7), (2, 4), (3, 3), (2, 0), (2, 1), (2, 2) ],
        #     [ (3, 1), (3, 0), (3, 2), (4, 3), (3, 4), (3, 5), (5, 4), (7, 7) ]
        # ]
        self.turn = 1
        self.tower_to_move = -1
        self.tower_to_move_coord = None

        self.last_player_before_blocked = None
        self.blocked_coord_list = None

        self.is_win = False

    # Defines all possible movements on the board.
    def possible_moves(self):
        result = []
        forward = 1 if self.turn == 0 else -1
        row, col = self.tower[self.turn][self.tower_to_move]
        cur_tower_list = self.tower[0] + self.tower[1]

        # strait moves
        for i in range(1, 8):
            if forward == 1 and row + i * forward > 7: break
            elif forward == -1 and row + i * forward < 0: break
            if (row + i * forward, col) in cur_tower_list: break
            else: result.append((row + i * forward, col))

        # left diagonal moves
        for i in range(1, 8):
            if forward == 1 and (row + i * forward > 7 or col + i * forward > 7): break
            elif forward == -1 and (row + i * forward < 0 or col + i * forward < 0): break
            if (row + i * forward, col + i * forward) in cur_tower_list: break
            else: result.append((row + i * forward, col + i * forward))

        # right diagonal moves
        for i in range(1, 8):
            if forward == 1 and (row + i * forward > 7 or col - i * forward < 0): break
            elif forward == -1 and (row + i * forward < 0 or col - i * forward > 7): break
            if (row + i * forward, col - i * forward) in cur_tower_list: break
            else: result.append((row + i * forward, col - i * forward))
        
        # when tower is blocked
        if not result: result.append((row, col))

        return result
    
    # Define how to update the board after the player moves.
    def make_move(self, move):
        print(move,"move 가 이거일떄")
        print(self.turn, "make_move실행")
        self.tower[self.turn][self.tower_to_move] = move

        # check if any player win
        if not self.is_over():
            print(self.turn,"set_next_turn실행")
            self.set_next_turn(move)
        
        # check if next player is blocked
        if self.is_blocked():
            # check for deadlock
            if self.is_deadlock(self.turn, self.tower_to_move, self.tower_to_move_coord):
                self.is_win = True
    
    def is_blocked(self, coord=None, turn=None):
        coord = coord if coord is not None else self.tower_to_move_coord
        turn = turn if turn is not None else self.turn
        forward = 1 if turn == 0 else -1
        towers = self.tower[0] + self.tower[1]

        c1 = (coord[0] + forward, coord[1]) in towers
        c2 = (coord[0] + forward, 0 if coord[1] - 1 < 0 else coord[1] - 1) in towers
        c3 = (coord[0] + forward, 7 if coord[1] + 1 > 7 else coord[1] + 1) in towers

        if c1 and c2 and c3: return True
        else: return False

    def is_deadlock(self, turn, tower_to_move, tower_to_move_coord):
        turn = 1 if turn == 0 else 0
        tower_to_move = BOARD[tower_to_move_coord[0]][tower_to_move_coord[1]]
        tower_to_move_coord = self.tower[turn][tower_to_move]
        if self.is_blocked(tower_to_move_coord, turn):
            if tower_to_move_coord not in self.blocked_coord_list:
                self.blocked_coord_list.append(tower_to_move_coord)
                self.is_deadlock(turn, tower_to_move, tower_to_move_coord)
            # deadlock occurs!
            else: return True
        else: return False
    
    def set_next_turn(self, coord):
        self.blocked_coord_list = [ coord ]
        self.last_player_before_blocked = self.turn

        # switch turn
        self.turn = 1 if self.turn == 0 else 0

        # move tower
        self.tower_to_move = BOARD[coord[0]][coord[1]]
        self.tower_to_move_coord = self.tower[self.turn][self.tower_to_move]
        self.first_move = False

    # check if the game is over
    def is_over(self):        
        return (
            self.is_win
            or self.win(self.current_player - 1)
            or self.win(self.opponent_index - 1)
        )

    def scoring(self):
        opp_won = self.win(self.opponent_index - 1)
        i_won = self.win(self.current_player - 1)
        if opp_won and not i_won:
            return 100
        elif i_won and not opp_won:
            return -100
        else:
            return 0

    def win(self, who):
        end_line_index = 7 if who == 0 else 0
        for i in self.tower[who]:
            if i[0] == end_line_index:
                return True
        return False

    def select_tower_to_move_first(self, coord):
        for i, t in enumerate(self.tower[self.turn]):
            if coord == t:
                self.tower_to_move = i
                self.tower_to_move_coord = coord

    def reset_tower_to_move_first(self):
        self.tower_to_move = -1
            

def draw_grid():
    for i in range(8): # index of row
        for j in range(8): # index of col
            # locate a rect at i-th row and j-th col
            rect = pygame.Rect(
                (j * WIDTH/8, i * HEIGHT/8), (WIDTH/8, HEIGHT/8))
            pygame.draw.rect(screen, COLORS[BOARD[i][j]], rect, width=0)

def draw_tower(game):
    for p in range(2): # player (0 or 1)
        for i in range(8): # each tower
            center = (game.tower[p][i][1] * WIDTH/8 + WIDTH/16,
                        game.tower[p][i][0] * HEIGHT/8 + HEIGHT/16)
            pygame.draw.circle(screen, COLORS[i], center, WIDTH/16, width=0)
            pygame.draw.circle(screen, PLAYERCOLOR[p], center, WIDTH/16, width=2)

def _message_margin(txt, backgr, textcol):
    rect = pygame.Rect( (0, HEIGHT), (WIDTH, MESSAGE_MARGIN) )
    pygame.draw.rect(screen, backgr, rect)
    pos_text = font.render(txt, True, textcol)
    pos_rect = pos_text.get_rect()
    pos_rect.center = ( WIDTH/2, HEIGHT + MESSAGE_MARGIN/2 )
    screen.blit(pos_text, pos_rect)

def msg_turn(game):
    title = "Win" if game.is_over() else "Turn"

    if game.turn == 0: # turn of blacks
        txt = "{}: {}".format(title, 'black')
        _message_margin(txt, BLACK, WHITE)
    else:
        txt = "{}: {}".format(title, 'white')
        _message_margin(txt, WHITE, BLACK)

def msg_blocked(game):
    if game.turn == 0: # turn of blacks
        txt = "Blocked: next turn -> {} in {} secs".format('white', BLOCK_DELAY)
        _message_margin(txt, BLACK, WHITE)
    else:
        txt = "Blocked: next turn -> {} in {} secs".format('black', BLOCK_DELAY)
        _message_margin(txt, WHITE, BLACK)

def highlight_cell(game):
    if game.tower_to_move >= 0:
        rect = pygame.Rect(
                    (game.tower_to_move_coord[1] * WIDTH/8, 
                    game.tower_to_move_coord[0] * HEIGHT/8),
                    (WIDTH/8, HEIGHT/8))
        pygame.draw.rect(screen, PLAYERCOLOR[game.turn], rect, width=2)

def calc_coord(pos):
    return (int(pos[1] / (HEIGHT/8)), int(pos[0] / (WIDTH/8)))

def draw_board(game):
    draw_grid()
    draw_tower(game)
    msg_turn(game)
    highlight_cell(game)
    pygame.display.update()

if __name__ == '__main__':
    kamisado = Kamisado([Human_Player(), AI_Player(Negamax(1))])
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                coord = calc_coord(pygame.mouse.get_pos())

                if kamisado.tower_to_move < 0:
                    kamisado.select_tower_to_move_first(coord)
                    continue

                if kamisado.first_move and kamisado.tower_to_move_coord == coord:
                    kamisado.reset_tower_to_move_first()
                
                # MY TURN
                elif coord in kamisado.possible_moves():
                    kamisado.play_move(coord)

                    if kamisado.is_over():
                        kamisado.tower_to_move_coord = coord

                    # AI TURN
                    else:
                        draw_board(kamisado)

                        pygame.time.delay(500)
                        ai_move = kamisado.get_move()
                        kamisado.play_move(ai_move)

                        if kamisado.is_over():
                            kamisado.tower_to_move_coord = ai_move
                        
        draw_board(kamisado)

(3, 3) move 가 이거일떄
1 make_move실행
1 set_next_turn실행
(1, 0) move 가 이거일떄
0 make_move실행
0 set_next_turn실행
(2, 0) move 가 이거일떄
0 make_move실행
0 set_next_turn실행
(3, 0) move 가 이거일떄
0 make_move실행
0 set_next_turn실행
(4, 0) move 가 이거일떄
0 make_move실행
0 set_next_turn실행
(5, 0) move 가 이거일떄
0 make_move실행
0 set_next_turn실행
(6, 0) move 가 이거일떄
0 make_move실행
0 set_next_turn실행
(1, 1) move 가 이거일떄
0 make_move실행
0 set_next_turn실행
(2, 2) move 가 이거일떄
0 make_move실행
0 set_next_turn실행
(1, 0) move 가 이거일떄
0 make_move실행
0 set_next_turn실행


SystemExit: 